In [2]:
### 问题二
## 2.优化算法，用字典求解

# -------------------------------------------------- 数据预处理 --------------------------------------------------
# Q2_3 数据预处理 得到字典 d_reflect

import numpy as np
import pandas as pd
from pyqubo import Array, Constraint, Placeholder
from openjij import SQASampler

# 矩阵 shape
N1 = 10
N2 = 3

# 读取附件 csv
ht = pd.read_csv('data_100.csv') 
hv = ht.values
h1 = []
t1 = []

# 建立 t, h array
for o in range(hv.shape[1]):
    if (o%2 == 0):
        t1.append(hv[:,o])
    else:
        h1.append(hv[:,o])

t = np.array([np.empty(N1)]*N2)
h = np.array([np.empty(N1)]*N2)

for k in range(N2):
    for l in range(N1):
        t[k][l] = t1[k][l]
        h[k][l] = h1[k][l]

d = {}
dic = {}

td = []
hd = []
ad = []

tmax, tmin = 0, 0
hmax, hmin = 0, 0

# 建立 hash 表
for i in range(N2):
    for j in range(N1):
        td.append(t[i, j])
        hd.append(h[i, j])
        d.setdefault((i, j), [t[i][j], h[i][j]])
        ad.append([t[i][j], h[i][j]])
        
def get_key(val):
    for key, value in d.items():
         if val == value:
             return key


tmax, tmin = np.max(td), np.min(td)
hmax, hmin = np.max(hd), np.min(hd)


tdt = {}
for i in np.arange(tmin, tmax+0.01, 0.01):
    tdt.setdefault(round(i, 2), [])

  
hdt = {}
for j in np.arange(hmin, hmax+0.001, 0.001):
    hdt.setdefault(round(j, 3), [])
  

for q in ad:
    tdt[q[0]].append(q[1])
    hdt[q[1]].append(q[0])
    


# 建立一个保留了相对位置信息、删去多余行列的矩阵L2，每一个元素的行列位置p,q（需要能够对应到原始data_100中的位置i，j）
L2_1 = np.zeros(N1)
L2_t = np.zeros((N2, N1))
L2_h = np.zeros((N2, N1))

# 对每个 t 求最小的 h，将筛选后的对应值保存在 L2_t & L2_h 矩阵中
for i in range(int((tmax-tmin)/0.01)+1):
    a = list(tdt.values())[i]
    # 数据不完全时需考虑考虑空 value
    if a != []:
        final_t_and_h = [round(i*0.01+tmin, 2), np.min(a)]
        dic.setdefault(tuple(final_t_and_h), get_key(final_t_and_h))
        
        a = get_key(final_t_and_h)[0]
        b = get_key(final_t_and_h)[1]

        L2_t[a][b] = final_t_and_h[0]
        L2_h[a][b] = final_t_and_h[1]
    
# 对 L2_h 中同样的 h，比对相同坐标下 t 的大小
# 若 h 相同，且 L2_t[x1][y1] > L2_t[x2][y2]（其中 x1, y1 为将要进行比对的坐标，x2, y2 为原先坐标）
# 则让 x2, y2 坐标对应下的 L2_h[x2][y2] 和 L2_t[x2][y2] 的值为 0 
d_same_h_t = {}
for i in range(N2):
    for j in range(N1):
        if L2_h[i][j] != 0:
            if L2_h[i][j] not in d_same_h_t:
                d_same_h_t.setdefault(L2_h[i][j], [(i, j), L2_t[i][j]])
            else:
                if L2_t[i][j] > d_same_h_t[L2_h[i][j]][1]:
                    x_point, y_point = d_same_h_t[L2_h[i][j]][0]
                    L2_t[x_point, y_point], L2_h[x_point, y_point] = 0, 0
                    d_same_h_t[L2_h[i][j]] = [(i, j), L2_t[i][j]]
                else:
                    L2_t[i, j], L2_h[i, j] = 0, 0

d_l2 = {}
d_index = {}

# Hash 记录原先非零值对应坐标
for i in range(N2):
    for j in range(N1):
        if L2_t[i][j] != 0:
            d_l2.setdefault((i, j), [L2_t[i][j], L2_h[i][j]])
            d_index.setdefault(L2_t[i][j], (i, j))


copy_L2_t = L2_t
copy_L2_h = L2_h

delrow = []
delcol = []

# 当某行值全为 0 时（加和为 0），删去该行； 当某列值全为 0 时（加和为 0），删去该列
for i in range(N2):
    if np.sum([L2_t[i][j] for j in range(N1)]) == 0:
        delrow.append(i)
        
for j in range(N1):
    if np.sum([L2_t[i][j] for i in range(N2)]) == 0:
        delcol.append(j)


copy_L2_t = np.delete(copy_L2_t, delrow, 0)
copy_L2_t = np.delete(copy_L2_t, delcol, 1)

copy_L2_h = np.delete(copy_L2_h, delrow, 0)
copy_L2_h = np.delete(copy_L2_h, delcol, 1)

# Hash 记录删去后对应值，进行反向检索
d_reflect = {}

for i in range(copy_L2_t.shape[0]):
    for j in range(copy_L2_t.shape[1]):
        if copy_L2_t[i][j] != 0:
            d_reflect.setdefault((i, j), d_index[copy_L2_t[i][j]])


# -------------------------------------------------- 数据预处理结束 --------------------------------------------------

# -------------------------------------------------- 计算 R_all(-L) --------------------------------------------------
# 贷款利息收益率
R = 0.08

# 构建 H_all 向量，从 hmin 到 hmax 进行值遍历，以 0.001 为步长
# 构建 T_all 向量，从 0.34(0.7^(3)) 到 tmax 进行值遍历，以 0.001 为步长
H_all = np.arange(hmin, hmax+0.001, 0.001)
T_all = np.arange(0.34, tmax+0.01, 0.01)

# R_all(-L) 为所有可能 t 与 h 对应最终收入的二维 Array
R_all = np.empty((len(H_all), len(T_all)))

# 计算 R_all
for i in range(len(H_all)):
    for j in range(len(T_all)):
        R_all[i][j] = R * T_all[j] *(1 - H_all[i]) - T_all[j] * H_all[i]

# -------------------------------------------------- R_all 计算完成 --------------------------------------------------

In [3]:
# -------------------------------------------------- 退火 --------------------------------------------------
# 读取 matrix shape
N1 = copy_L2_t.shape[1]
N2 = copy_L2_h.shape[0]

# 约束系数 M
M = 2

h1 = []
t1 = []

# 建立 t, h Array
for o in range(hv.shape[1]):
    if (o%2 == 0):
        t1.append(hv[:,o])
    else:
        h1.append(hv[:,o])

t = np.array([np.empty(N1)]*N2)
h = np.array([np.empty(N1)]*N2)

for k in range(N2):
    for l in range(N1):
        t[k][l] = t1[k][l]
        h[k][l] = h1[k][l]

# 建立 x array
x = Array.create('x', (N2, N1), 'BINARY')

# # 定义约束条件和目标函数
# 约束条件 1:
# 每张卡最多选择 1 个阈值（在每一纵列中，至多出现一个 1 ，可以全为 0）
a = []
for p in range(N2):
    for n in range(N1):
        for m in range(n+1, N1):
            a.append(x[p][n]*x[p][m])
H1 = np.sum(a)

# 计算 H0(Loss)
d_L = {}

# 将每个 x 中 (x, 0) 的值设为 -M ( 59 ≈ (0.88+0.87)/3 )
for zero in range(60):
    d_L.setdefault((round(zero*0.001, 3), 0), -M)
    

for i in range(R_all.shape[0]):
    for j in range(R_all.shape[1]):
        d_L.setdefault((round(hmin+i*0.001, 3), round(0.34+j*0.01, 2)), round(R_all[i][j], 7))
        
H0 = np.sum([d_L[(round((copy_L2_h[0][j] + copy_L2_h[1][q] + copy_L2_h[2][r])/3, 3),\
    round(copy_L2_t[0][j] * copy_L2_t[1][q] * copy_L2_t[2][r], 2))] * x[0][j] * x[1][q] * x[2][r] \
    for j in range(N1) for q in range(N1) for r in range(N1)])

# 目标函数
H = -H0 + Placeholder('M') * Constraint(H1, label='H1')
model = H.compile()


feed_dict = {'M': M}

# 求出 QUBO 矩阵
qubo, offset = model.to_qubo(feed_dict=feed_dict)


# Sample(使用 SQASampler 模拟退火)
sampler = SQASampler(num_sweeps=3000)

# num_reads
R = 1500

sampleset = sampler.sample_qubo(qubo, num_reads=R)

# -------------------------------------------------- 退火结束 --------------------------------------------------

# Save to ./xx/xxx_x_xxx.csv
save_path = './save_csv/ans_2_2_{}iter_1.csv'.format(R)

save = pd.DataFrame(sampleset)
save.to_csv(save_path)


# 反向 Hash 搜索
index_card = []

for i, row in save.iloc[[0],:].iterrows():
    for j in range(len(row)):
        if row[j] == 1:
            outrow = row.index[j].split('x')
            if len(outrow)==2:
                index_card.append(tuple(eval(row.index[j].split('x')[1].replace('][', ','))))
            
d_reflect_keys = d_reflect.keys()
d_ans = []
d_f = {}
for i in d_reflect.values():
    t_n, h_n = L2_t[i], L2_h[i]
    d_ans.append(get_key([t_n, h_n]))
d_f = dict(zip(d_reflect_keys, d_ans))

ans_1 = d_f[index_card[0]]
ans_2 = d_f[index_card[1]]
ans_3 = d_f[index_card[2]]

# 输出结果
print('第一张信用评分卡选择第 {} 个阈值'.format(ans_1[1]+1))
print('第二张信用评分卡选择第 {} 个阈值'.format(ans_2[1]+1))
print('第三张信用评分卡选择第 {} 个阈值'.format(ans_3[1]+1))

第一张信用评分卡选择第 8 个阈值
第二张信用评分卡选择第 1 个阈值
第三张信用评分卡选择第 2 个阈值
